In [1]:
import numpy as np
import pandas as pd
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import math
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import os
import glob

from sklearn.cluster import DBSCAN, KMeans

In [2]:

data_path='/home/madnisal/Documents/ML_Project/datasets/'
training_df = pd.read_csv(data_path+'train.csv', index_col="tripid")

In [3]:
def dist_from_coordinates(lat1, lon1, lat2, lon2):
  R = 6371  # Earth radius in km

  #conversion to radians
  d_lat = np.radians(lat2-lat1)
  d_lon = np.radians(lon2-lon1)

  r_lat1 = np.radians(lat1)
  r_lat2 = np.radians(lat2)

  #haversine formula
  a = np.sin(d_lat/2.) **2 + np.cos(r_lat1) * np.cos(r_lat2) * np.sin(d_lon/2.)**2

  haversine = 2 * R * np.arcsin(np.sqrt(a))

  return haversine

In [4]:
training_df['pickup_time'] = pd.to_datetime(training_df['pickup_time'], format="%m/%d/%Y %H:%M")
training_df['drop_time'] = pd.to_datetime(training_df['drop_time'], format="%m/%d/%Y %H:%M")

In [5]:
training_df = training_df.assign(timeOfDay=pd.cut(training_df.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [6]:
training_df.loc[training_df['timeOfDay'] == 'day', 'isNormalCharge'] = 1
training_df.loc[training_df['timeOfDay'] != 'day', 'isNormalCharge'] = 0

In [7]:
durations = []
for index,row in training_df.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = np.nan
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

training_df.insert(4,"time_dif",durations)

In [8]:
new_column = []                    #empty column for distance
for index,row in training_df.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

training_df.insert(4,"distance",new_column)

In [9]:
training_df['time_driven'] = training_df['duration']  - training_df['meter_waiting']

In [10]:
chargeperhours = []
for index,row in training_df.iterrows():
    if(row['meter_waiting'] == 0):
        chargeperhour = np.nan
    else:
        chargeperhour = (row['meter_waiting_fare'] / row['meter_waiting'] * 3600)
    chargeperhours.append(chargeperhour)

training_df.insert(4,'charge_per_hour',chargeperhours)


In [11]:
training_df['driving_fare'] = training_df['fare']  - training_df['meter_waiting_fare'] - training_df['additional_fare']

In [12]:
avgspeeds = []
for index,row in training_df.iterrows():
    if(row['time_driven'] == 0):
        avgspeed = np.nan
    else:
        avgspeed = (row['distance'] / row['time_driven'] * 3600)
    avgspeeds.append(avgspeed)

training_df.insert(4,"avg_speed",avgspeeds)



In [13]:
costsperkm = []
for index,row in training_df.iterrows():
    if row['distance'] == 0:
        costperkm = np.nan
            
    else:
        costperkm = (row['driving_fare'] / row['distance'])
    costsperkm.append(costperkm)

training_df.insert(4,"cost_per_km",costsperkm)

In [14]:
training_df = training_df.replace({'label': {'incorrect': 0, 'correct' : 1}})

<h2>Geolocation clustering</h2>

In [15]:
pickup_locations = training_df[['pick_lat','pick_lon']]
drop_locations = training_df[['drop_lat','drop_lon']]

In [16]:
kmeans = KMeans(n_clusters = 5, init ='k-means++')
kmeans.fit(pickup_locations)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [17]:
training_df['pickup_cluster_label'] = kmeans.predict(pickup_locations)

In [20]:
kmeansdrop = KMeans(n_clusters = 5, init ='k-means++')
kmeansdrop.fit(drop_locations)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [21]:
training_df['drop_cluster_label'] = kmeansdrop.predict(drop_locations)

<h2>Selecting the features</h2>

In [22]:
training_df.columns

Index(['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'cost_per_km', 'avg_speed', 'charge_per_hour', 'distance', 'time_dif',
       'meter_waiting_till_pickup', 'pickup_time', 'drop_time', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'fare', 'label', 'timeOfDay',
       'isNormalCharge', 'time_driven', 'driving_fare', 'pickup_cluster_label',
       'drop_cluster_label'],
      dtype='object')

In [23]:
training_columns = ['meter_waiting','meter_waiting_fare','fare','additional_fare', 'distance','cost_per_km', 'avg_speed',  'time_dif','time_driven', 'charge_per_hour', 'driving_fare', 'isNormalCharge','drop_cluster_label','pickup_cluster_label']

In [24]:
training_columns = ['meter_waiting','meter_waiting_fare','fare','additional_fare', 'distance','cost_per_km', 'avg_speed',  'time_dif','time_driven', 'charge_per_hour', 'driving_fare', 'isNormalCharge','pick_lat','pick_lon','drop_lat','drop_lon','drop_cluster_label','pickup_cluster_label']

In [25]:
target_column = ['label']

In [26]:
training_df = training_df.drop(190167541)

In [27]:
x = training_df[training_columns].values
y = training_df[target_column].values

In [28]:
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [29]:
training_df['label'].value_counts()

1    15494
0     1681
Name: label, dtype: int64

<h2>Choosing the best model</h2>

In [30]:
def focal_loss_lgb_eval_error(y_true, y_pred, alpha=.25, gamma=2.):
    a,g = alpha, gamma
    p = 1/(1+np.exp(-y_pred))
    loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
    return 'focal_loss', np.mean(loss), False

In [31]:
from sklearn.metrics import f1_score
def evaluate_macroF1_lgb(y_true, y_pred):  
    y_hat = np.where(y_pred < 0.5, 0, 1) 
    f1 = f1_score(y_true, y_hat, average='macro')
    return ('macroF1', f1, True) 

In [32]:
gkf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [33]:
model = lightgbm.LGBMClassifier(class_weight={0:1.5,1:1}, learning_rate=0.1)

In [34]:
for training_index, testing_index in gkf.split(X=x, y=y):
    x_train_fold, y_train_fold = x[training_index], y[training_index]
    x_test_fold, y_test_fold = x[testing_index], y[testing_index]
    model.fit(x_train_fold, y_train_fold, eval_set=(x_test_fold,y_test_fold),eval_metric = lambda y_true, y_pred: [evaluate_macroF1_lgb(y_true,y_pred)])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.28951	valid_0's macroF1: 0.474268
[2]	valid_0's binary_logloss: 0.266245	valid_0's macroF1: 0.474268
[3]	valid_0's binary_logloss: 0.248837	valid_0's macroF1: 0.474268
[4]	valid_0's binary_logloss: 0.235365	valid_0's macroF1: 0.474268
[5]	valid_0's binary_logloss: 0.224279	valid_0's macroF1: 0.691565
[6]	valid_0's binary_logloss: 0.214593	valid_0's macroF1: 0.743383
[7]	valid_0's binary_logloss: 0.206098	valid_0's macroF1: 0.79792
[8]	valid_0's binary_logloss: 0.198465	valid_0's macroF1: 0.810983
[9]	valid_0's binary_logloss: 0.192515	valid_0's macroF1: 0.820103
[10]	valid_0's binary_logloss: 0.186296	valid_0's macroF1: 0.826652
[11]	valid_0's binary_logloss: 0.181639	valid_0's macroF1: 0.837845
[12]	valid_0's binary_logloss: 0.177881	valid_0's macroF1: 0.839369
[13]	valid_0's binary_logloss: 0.174443	valid_0's macroF1: 0.837385
[14]	valid_0's binary_logloss: 0.171348	valid_0's macroF1: 0.840413
[15]	valid_0's binary_logloss: 0.16799	valid_0's macroF1: 0

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[20]	valid_0's binary_logloss: 0.164609	valid_0's macroF1: 0.832602
[21]	valid_0's binary_logloss: 0.163247	valid_0's macroF1: 0.832409
[22]	valid_0's binary_logloss: 0.162132	valid_0's macroF1: 0.830702
[23]	valid_0's binary_logloss: 0.160998	valid_0's macroF1: 0.833524
[24]	valid_0's binary_logloss: 0.160235	valid_0's macroF1: 0.832678
[25]	valid_0's binary_logloss: 0.158948	valid_0's macroF1: 0.835656
[26]	valid_0's binary_logloss: 0.158255	valid_0's macroF1: 0.837136
[27]	valid_0's binary_logloss: 0.157739	valid_0's macroF1: 0.837136
[28]	valid_0's binary_logloss: 0.157354	valid_0's macroF1: 0.837987
[29]	valid_0's binary_logloss: 0.156838	valid_0's macroF1: 0.835656
[30]	valid_0's binary_logloss: 0.156197	valid_0's macroF1: 0.840933
[31]	valid_0's binary_logloss: 0.155716	valid_0's macroF1: 0.840079
[32]	valid_0's binary_logloss: 0.155229	valid_0's macroF1: 0.840079
[33]	valid_0's binary_logloss: 0.154674	valid_0's macroF1: 0.838611
[34]	valid_0's binary_logloss: 0.154503	valid_0'

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[6]	valid_0's binary_logloss: 0.220618	valid_0's macroF1: 0.750002
[7]	valid_0's binary_logloss: 0.212485	valid_0's macroF1: 0.771408
[8]	valid_0's binary_logloss: 0.205891	valid_0's macroF1: 0.787239
[9]	valid_0's binary_logloss: 0.200296	valid_0's macroF1: 0.804069
[10]	valid_0's binary_logloss: 0.195181	valid_0's macroF1: 0.809109
[11]	valid_0's binary_logloss: 0.189827	valid_0's macroF1: 0.819008
[12]	valid_0's binary_logloss: 0.1852	valid_0's macroF1: 0.824594
[13]	valid_0's binary_logloss: 0.181538	valid_0's macroF1: 0.82445
[14]	valid_0's binary_logloss: 0.177838	valid_0's macroF1: 0.828152
[15]	valid_0's binary_logloss: 0.175056	valid_0's macroF1: 0.829706
[16]	valid_0's binary_logloss: 0.172212	valid_0's macroF1: 0.833466
[17]	valid_0's binary_logloss: 0.169503	valid_0's macroF1: 0.834579
[18]	valid_0's binary_logloss: 0.168013	valid_0's macroF1: 0.833725
[19]	valid_0's binary_logloss: 0.166	valid_0's macroF1: 0.836723
[20]	valid_0's binary_logloss: 0.16475	valid_0's macroF1: 

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[14]	valid_0's binary_logloss: 0.16819	valid_0's macroF1: 0.833189
[15]	valid_0's binary_logloss: 0.164845	valid_0's macroF1: 0.835413
[16]	valid_0's binary_logloss: 0.162463	valid_0's macroF1: 0.841289
[17]	valid_0's binary_logloss: 0.160267	valid_0's macroF1: 0.840654
[18]	valid_0's binary_logloss: 0.15802	valid_0's macroF1: 0.841289
[19]	valid_0's binary_logloss: 0.156122	valid_0's macroF1: 0.843416
[20]	valid_0's binary_logloss: 0.153989	valid_0's macroF1: 0.844032
[21]	valid_0's binary_logloss: 0.152682	valid_0's macroF1: 0.842541
[22]	valid_0's binary_logloss: 0.150773	valid_0's macroF1: 0.842541
[23]	valid_0's binary_logloss: 0.149291	valid_0's macroF1: 0.844909
[24]	valid_0's binary_logloss: 0.147846	valid_0's macroF1: 0.847876
[25]	valid_0's binary_logloss: 0.146623	valid_0's macroF1: 0.846395
[26]	valid_0's binary_logloss: 0.145795	valid_0's macroF1: 0.847876
[27]	valid_0's binary_logloss: 0.144532	valid_0's macroF1: 0.84847
[28]	valid_0's binary_logloss: 0.143758	valid_0's m

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[19]	valid_0's binary_logloss: 0.163975	valid_0's macroF1: 0.842168
[20]	valid_0's binary_logloss: 0.162363	valid_0's macroF1: 0.843416
[21]	valid_0's binary_logloss: 0.160501	valid_0's macroF1: 0.848758
[22]	valid_0's binary_logloss: 0.159002	valid_0's macroF1: 0.84847
[23]	valid_0's binary_logloss: 0.157522	valid_0's macroF1: 0.84847
[24]	valid_0's binary_logloss: 0.156158	valid_0's macroF1: 0.846719
[25]	valid_0's binary_logloss: 0.155062	valid_0's macroF1: 0.846996
[26]	valid_0's binary_logloss: 0.154442	valid_0's macroF1: 0.84847
[27]	valid_0's binary_logloss: 0.153071	valid_0's macroF1: 0.850222
[28]	valid_0's binary_logloss: 0.151982	valid_0's macroF1: 0.853109
[29]	valid_0's binary_logloss: 0.151352	valid_0's macroF1: 0.855973
[30]	valid_0's binary_logloss: 0.150636	valid_0's macroF1: 0.857735
[31]	valid_0's binary_logloss: 0.14965	valid_0's macroF1: 0.859161
[32]	valid_0's binary_logloss: 0.149719	valid_0's macroF1: 0.857397
[33]	valid_0's binary_logloss: 0.14862	valid_0's mac

In [35]:
predicted_y = model.predict(x_test)
train_pred_y = model.predict(x)

In [36]:
print(); print(metrics.classification_report(y,train_pred_y))
print(); print(metrics.confusion_matrix(y, train_pred_y))

print(); print(metrics.classification_report(y_test, predicted_y))
print(); print(metrics.confusion_matrix(y_test, predicted_y))


              precision    recall  f1-score   support

           0       0.95      0.85      0.90      1345
           1       0.98      1.00      0.99     12395

    accuracy                           0.98     13740
   macro avg       0.97      0.92      0.94     13740
weighted avg       0.98      0.98      0.98     13740


[[ 1138   207]
 [   57 12338]]

              precision    recall  f1-score   support

           0       0.88      0.69      0.77       336
           1       0.97      0.99      0.98      3099

    accuracy                           0.96      3435
   macro avg       0.92      0.84      0.88      3435
weighted avg       0.96      0.96      0.96      3435


[[ 231  105]
 [  31 3068]]


<h2>Testing</h2>

In [37]:

test_set = pd.read_csv(data_path+'test.csv', index_col="tripid")


In [38]:
test_set['pickup_time'] = pd.to_datetime(test_set['pickup_time'], format="%m/%d/%Y %H:%M")
test_set['drop_time'] = pd.to_datetime(test_set['drop_time'], format="%m/%d/%Y %H:%M")

In [39]:
test_set = test_set.assign(timeOfDay=pd.cut(test_set.pickup_time.dt.hour,[-1, 8, 20, 24],labels=['dawn','day', 'night']))

In [40]:
test_set.loc[test_set['timeOfDay'] == 'day', 'isNormalCharge'] = 1
test_set.loc[test_set['timeOfDay'] != 'day', 'isNormalCharge'] = 0

In [41]:
durations = []
for index,row in test_set.iterrows():
  provided_duration = row['duration'] #first row of location.lat column here
  if math.isnan(provided_duration) or provided_duration <= 0 :
    time_dif = (row['drop_time'] - row['pickup_time']).seconds
    if(time_dif == 0):
        time_dif = np.nan
    durations.append(time_dif)
  else :  
    durations.append(provided_duration)

test_set.insert(4,"time_dif",durations)

In [42]:
new_column = []                    #empty column for distance
for index,row in test_set.iterrows():
  lat1 = row['pick_lat'] #first row of location.lat column here
  lon1 = row['pick_lon'] #first row of location.long column here
  lat2 = row['drop_lat'] #second row of location.lat column here
  lon2 = row['drop_lon'] #second row of location.long column here
  value = dist_from_coordinates(lat1, lon1, lat2, lon2)  #get the distance
  new_column.append(value)   #append the empty list with distance values

test_set.insert(4,"distance",new_column)

In [43]:
test_set['time_driven'] = test_set['duration']  - test_set['meter_waiting']

In [44]:
chargeperhours = []
for index,row in test_set.iterrows():
    if(row['meter_waiting'] == 0):
        chargeperhour = np.nan
    else:
        chargeperhour = (row['meter_waiting_fare'] / row['meter_waiting'] * 3600)
    chargeperhours.append(chargeperhour)

test_set.insert(4,'charge_per_hour',chargeperhours)


In [45]:
test_set['driving_fare'] = test_set['fare']  - test_set['meter_waiting_fare'] - test_set['additional_fare']

In [46]:
avgspeeds = []
for index,row in test_set.iterrows():
    if(row['time_driven'] == 0):
        avgspeed = np.nan    
    else:
        avgspeed = (row['distance'] / row['time_driven'] * 3600)
    avgspeeds.append(avgspeed)

test_set.insert(4,"avg_speed",avgspeeds)



In [47]:
costsperkm = []
for index,row in test_set.iterrows():
    if row['distance'] == 0:
        costperkm = np.nan
            
    else:
        costperkm = (row['driving_fare'] / row['distance'])
    costsperkm.append(costperkm)

test_set.insert(4,"cost_per_km",costsperkm)

In [48]:
test_set['pickup_cluster_label'] = kmeans.predict(test_set[['pick_lat','pick_lon']])
test_set['drop_cluster_label'] = kmeansdrop.predict(test_set[['drop_lat','drop_lon']])

In [49]:
test_features = test_set[training_columns]

In [50]:
test_features.isna().sum()

meter_waiting             0
meter_waiting_fare        0
fare                      0
additional_fare           0
distance                  0
cost_per_km              25
avg_speed                 9
time_dif                  0
time_driven               0
charge_per_hour         298
driving_fare              0
isNormalCharge            0
pick_lat                  0
pick_lon                  0
drop_lat                  0
drop_lon                  0
drop_cluster_label        0
pickup_cluster_label      0
dtype: int64

In [51]:
predicted_labels = model.predict(test_features)

In [52]:
predicted_labels_df = pd.DataFrame(predicted_labels )

In [53]:
sub_path =os.path.abspath(os.path.join(data_path+'/sample_submission.csv'))
submission_set = pd.read_csv(sub_path, index_col="tripid")

In [54]:
submission_set['prediction']= predicted_labels_df.values[:,0]

In [55]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [57]:
filename = '../../submissions/'+theNotebook+'/'+theNotebook+'_{%i}.csv'
dirname = '../../submissions/'+theNotebook
fileversion = 1

if not os.path.exists(dirname):
    os.makedirs(dirname)
while glob.glob(filename.replace('{%i}',str(fileversion))) :
    fileversion+=1
submission_set.to_csv(filename.replace('{%i}',str(fileversion)), index=True)
print("Completed!")

Completed!


In [56]:
submission_set['prediction'].value_counts()

1    8120
0     456
Name: prediction, dtype: int64